In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import optimus as Optimus

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
C:\Users\argenisleon\Anaconda3\lib\site-packages\statsmodels\base\wrapper.py:100: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  defaults=argspec[3])

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 


In [4]:
op= Optimus("dask", comm=True)

In [5]:
df = op.load.csv("data/foo.csv", sep=",", header='true', infer_schema='true', charset="UTF-8", null_value="None")

In [6]:
df.ext.display()

id 1 (int64) not nullable,firstName 2 (object) not nullable,lastName 3 (object) not nullable,billingId 4 (int64) not nullable,product 5 (object) not nullable,price 6 (int64) not nullable,birth 7 (object) not nullable,dummyCol 8 (object) not nullable
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [10]:
df.cols.median("*")

AttributeError: type object 'Cols' has no attribute 'percentile'

In [7]:
df.meta.get()

AttributeError: 'DataFrame' object has no attribute 'meta'

Dask DataFrame Structure:
                  id firstName lastName billingId product  price   birth dummyCol
npartitions=1                                                                    
               int64    object   object     int64  object  int64  object   object
                 ...       ...      ...       ...     ...    ...     ...      ...
Dask Name: from-delayed, 3 tasks


In [9]:
df.cols.rename("id", "id1").meta.get()

{'file_name': 'foo.csv',
 'transformations': {'actions': {'rename': {'id': 'id1'}}}}

In [8]:
df.ext.display()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [11]:
df.cols.remove_special_chars("lastName")

invalid escape sequence \d


DataFrame[id: int, firstName: string, lastName: string, billingId: int, product: string, price: int, birth: string, dummyCol: string]

In [9]:
df.ext.send()

Send!


In [10]:
from optimus.profiler.profiler import Profiler
p = Profiler()

df.cols.count_by_dtypes("*")

{'id': {'int': 19,
  'decimal': 0,
  'string': 0,
  'date': 0,
  'boolean': 0,
  'binary': 0,
  'array': 0,
  'object': 0,
  'null': 0,
  'missing': 0},
 'firstName': {'int': 0,
  'decimal': 0,
  'string': 19,
  'date': 0,
  'boolean': 0,
  'binary': 0,
  'array': 0,
  'object': 0,
  'null': 0,
  'missing': 0},
 'lastName': {'int': 0,
  'decimal': 0,
  'string': 19,
  'date': 0,
  'boolean': 0,
  'binary': 0,
  'array': 0,
  'object': 0,
  'null': 0,
  'missing': 0},
 'billingId': {'int': 19,
  'decimal': 0,
  'string': 0,
  'date': 0,
  'boolean': 0,
  'binary': 0,
  'array': 0,
  'object': 0,
  'null': 0,
  'missing': 0},
 'product': {'int': 0,
  'decimal': 0,
  'string': 19,
  'date': 0,
  'boolean': 0,
  'binary': 0,
  'array': 0,
  'object': 0,
  'null': 0,
  'missing': 0},
 'price': {'int': 19,
  'decimal': 0,
  'string': 0,
  'date': 0,
  'boolean': 0,
  'binary': 0,
  'array': 0,
  'object': 0,
  'null': 0,
  'missing': 0},
 'birth': {'int': 0,
  'decimal': 0,
  'string': 19,
 